In [0]:
%sh
pip install tweepy demoji spacy langdetect
python -m spacy download en_core_web_lg

In [0]:
import tweepy
import pandas as pd

# Retrieve credentials from Databricks Secrets
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAFZM6wEAAAAAlIrbJyX9QVn6NlxHVSQDyY9mJ3o%3Dwv6BsFyI5f2bz2t2LAS9FwKyMUe8VWbKVhB9cHrp4RSIQpGWc7'

# Initialize the client
client = tweepy.Client(bearer_token=bearer_token)

# Define your search query (e.g., search for tweets about 'Azure Databricks')
query = 'Cricket OR cricket OR #Cricket -is:retweet'

# Fetch tweets
try:    tweets = client.search_recent_tweets(query=query, max_results=10, tweet_fields=['created_at', 'text'])
except tweepy.TooManyRequests:
    print('Rate limit exceeded. Please wait and try again later.')
    tweets = None


# Convert to a list of dictionaries for easier processing
tweet_data = []
if tweets is not None and tweets.data:
    for tweet in tweets.data:
        tweet_data.append({"id": tweet.id, "created_at": tweet.created_at, "text": tweet.text})

# Create a Spark DataFrame for analysis
df = spark.createDataFrame(pd.DataFrame(tweet_data))
display(df)

In [0]:

import os
import time
import tweepy
from tweepy.errors import TooManyRequests, TweepyException

bearer_token = 'AAAAAAAAAAAAAAAAAAAAAJ8z6wEAAAAASCV6kM5WJUNm3huTbRWWjThBQh4%3Dt7uERFwhIigcVmbQHpkqhgoTv6QfZRWnaCrFIMb35WiZkaGatX'
if not bearer_token:
    raise RuntimeError("Missing TWITTER_BEARER_TOKEN environment variable.")

client = tweepy.Client(bearer_token=bearer_token)

# Keep the query narrower to reduce rate-limit risk.
query = """
(
  #cricket OR cricket OR #T20 OR T20 OR #ODI OR ODI
  OR #WorldCup OR WorldCup OR #IPL OR IPL
) lang:en -is:retweet
""".strip()

def get_one_tweet():
    try:
        # Make exactly one API call (one page).
        resp = client.search_recent_tweets(
            query=query,
            max_results=10,  # one page, 10-100 allowed
            tweet_fields=["id", "text", "created_at", "lang"]
        )
        if resp.data and len(resp.data) > 0:
            t = resp.data[0]
            print("At least one tweet retrieved:")
            print(f"[{t.id}] {t.created_at} | {t.text}")
        else:
            print("No tweets retrieved by the query.")
    except TooManyRequests as e:
        # Sleep until reset based on headers (precise), otherwise fallback ~15 min
        wait_secs = 15 * 60
        try:
            reset_epoch = int(e.response.headers.get("x-rate-limit-reset"))
            now = int(time.time())
            wait_secs = max(reset_epoch - now, 60)
        except Exception:
            pass
        print(f"Rate limit exceeded. Sleeping for ~{wait_secs/60:.1f} minutes until reset...")
        time.sleep(wait_secs)
        # Optional: retry once after the sleep
        try:
            resp = client.search_recent_tweets(
                query=query, max_results=10, tweet_fields=["id", "text", "created_at", "lang"]
            )
            if resp.data and len(resp.data) > 0:
                t = resp.data[0]
                print("At least one tweet retrieved after reset:")
                print(f"[{t.id}] {t.created_at} | {t.text}")
            else:
                print("No tweets retrieved by the query after reset.")
        except TooManyRequests:
            print("Still rate limited after waiting; try later or narrow the query.")
    except TweepyException as e:
        print(f"TweepyException: {e}")

if __name__ == "__main__":
    get_one_tweet()


In [0]:
# # Initialize Twitter API and get tweets
# import tweepy
# import time
# from tweepy.errors import TooManyRequests

# bearer_token = 'AAAAAAAAAAAAAAAAAAAAAJ8z6wEAAAAASCV6kM5WJUNm3huTbRWWjThBQh4%3Dt7uERFwhIigcVmbQHpkqhgoTv6QfZRWnaCrFIMb35WiZkaGatX'

# client = tweepy.Client(bearer_token=bearer_token)
# query = (
#     '(#cricket OR #Cricket OR cricket OR Cricket OR #ICC OR ICC '
#     'OR #CWC OR CWC OR #T20 OR T20 OR #ODI OR ODI '
#     'OR #TestCricket OR TestCricket OR #WorldCup OR WorldCup '
#     'OR #BCCI OR BCCI OR #IPL OR IPL '
#     'OR #Ashes OR Ashes OR #INDvAUS OR #INDvENG OR #INDvPAK '
#     'OR #ENGvAUS OR #AUSvPAK OR #INDvsAUS OR #INDvsENG OR #INDvsPAK '
#     'OR #ViratKohli OR ViratKohli OR #RohitSharma OR RohitSharma '
#     'OR #MSDhoni OR MSDhoni OR #Sachin OR Sachin '
#     'OR #KaneWilliamson OR KaneWilliamson OR #JoeRoot OR JoeRoot '
#     'OR #ICC OR ICC OR #CWC OR CWC OR #T20 OR T20 OR #ODI OR ODI '
#     'OR #TestCr OR #T20WorldCup OR T20WorldCup'
#     'OR #SteveSmith OR SteveSmith OR #BabarAzam OR BabarAzam) lang:en -is:retweet'
# )

# tweets = tweepy.Paginator(
#     client.search_recent_tweets,
#     query,
#     max_results=100
# ).flatten(limit=10000)

# # Check if tweets are being produced
# try:
#     first_tweet = next(tweets)
#     print("At least one tweet retrieved:", first_tweet.text)
# except TooManyRequests:
#     print("Rate limit exceeded. Please wait and try again.")
#     time.sleep(15 * 60)  # Sleep for 15 minutes
#     first_tweet = next(tweets)
#     print("At least one tweet retrieved:", first_tweet.text)
# except StopIteration:
#     print("No tweets retrieved by the query.")

In [0]:
import pandas as pd
import time
import tweepy

tweetList = []
max_retries = 5
backoff_factor = 2

retries = 0
while retries < max_retries:
    try:
        for tweet in tweets:
            tweetList.append(tweet.text)
        break
    except tweepy.TooManyRequests as e:
        wait_time = getattr(e, 'retry_after', backoff_factor ** retries)
        print(f"Rate limit hit. Sleeping for {wait_time} seconds before retrying...")
        time.sleep(wait_time)
        retries += 1

if retries == max_retries:
    print("Max retries reached. Some tweets may be missing.")

df = pd.DataFrame(tweetList, columns=["text"])

In [0]:
# import pandas as pd
# import time
# import tweepy

# tweetList = []
# max_retries = 5
# backoff_factor = 2

# # Assuming 'tweets' is a Tweepy Paginator or iterable that may raise TooManyRequests

# retries = 0
# while retries < max_retries:
#     try:
#         for tweet in tweets:
#             tweetList.append(tweet.text)
#         break  # Exit retry loop if successful
#     except tweepy.TooManyRequests as e:
#         wait_time = getattr(e, 'retry_after', backoff_factor ** retries)
#         print(f"Rate limit hit. Sleeping for {wait_time} seconds before retrying...")
#         time.sleep(wait_time)
#         retries += 1

# if retries == max_retries:
#     print("Max retries reached. Some tweets may be missing.")

# df = pd.DataFrame(tweetList, columns=["text"])
# # display(df)

In [0]:
import demoji
import re
 
def cleanTweetText(tweets):
  tweetsText = []
  for i in tweets:
    # remove emojis
    tweet = demoji.replace(i)
    # remove URLs
    tweetNoURL = re.sub(r'http\S+', '', tweet)
    tweetsText.append(tweetNoURL)
  return tweetsText
 
df["text"] = cleanTweetText(df["text"])

In [0]:
import spacy
import string
from spacy.tokenizer import Tokenizer
import string
from string import digits
 
nlp = spacy.load("en_core_web_lg")
tokenizer = Tokenizer(nlp.vocab)
punct = str.maketrans('', '', string.punctuation)
remove_digits = str.maketrans('', '', digits)
 
def getTokens(text):
  tokens = []
  for doc in tokenizer.pipe(text):
    doc_tokens = []    
    for token in doc:
      if token.text.lower().startswith('@') or token.text.lower().startswith('\\') or '_' in token.text.lower() or 'rt' in token.text.lower() or '‰' in token.text.lower():
        continue
      token_no_punc = str(token.text).translate(punct).translate(remove_digits)
      if len(token_no_punc)>1:
        doc_tokens.append(token_no_punc)
    tokens.append(doc_tokens)
  return tokens
 
df["tokens_as_str"] = [' '.join(map(str, i)) for i in getTokens(df["text"])]

In [0]:
import numpy as np

df = df.replace(
    r'^\s*$',
    np.nan,
    regex=True
)
df = df.dropna()
df = df.drop_duplicates(
    subset=['text']
)

if not df.empty:
    display(df)
else:
    print("DataFrame is empty after cleaning.")

In [0]:
if not df.empty:
    updates = spark.createDataFrame(df)
    updates.createOrReplaceGlobalTempView("updates")
else:
    print("DataFrame is empty. No Spark DataFrame created.")

In [0]:
write_format = 'delta'
save_path = '/mnt/FileStore/bronze/tweets.delta'
table_name = 'bronze.tweeter_bronze.tweeter'
 
# Create the table if it does not already exist.
spark.sql("CREATE TABLE IF NOT EXISTS " + table_name + " USING DELTA LOCATION '" + save_path + "'")

In [0]:

import sys, platform
print("Python:", sys.version)
import numpy as np
print("NumPy:", np.__version__)


In [0]:

# Use one set of pins; these work well with spaCy/thinc native extensions
%pip install --upgrade --force-reinstall --no-cache-dir \
  "numpy>=1.24,<2.0" \
  "spacy>=3.6,<3.8" \
  "thinc>=8.1,<9.0"


In [0]:

# Create a directory in DBFS
dbutils.fs.mkdirs("dbfs:/FileStore/thinc")

# Use /dbfs for local-like access from Python
import os
path = "/dbfs/FileStore/thinc"
print("Exists:", os.path.exists(path))


In [0]:

import sys, platform
print("Python:", sys.version)
import numpy as np
print("NumPy:", np.__version__)


In [0]:

%pip install --upgrade --force-reinstall --no-cache-dir \
  "numpy>=1.24,<2.0" \
  "thinc>=8.1,<9.0" \
  "spacy>=3.6,<3.8"


In [0]:
dbutils.library.restartPython()

In [0]:

import numpy as np, thinc, spacy
print("NumPy:", np.__version__)
print("thinc:", thinc.__version__)
print("spaCy:", spacy.__version__)
import subprocess, sys
subprocess.run([sys.executable, "-m", "spacy", "validate"])


In [0]:

dbutils.fs.mkdirs("dbfs:/FileStore/thinc")
path = "/dbfs/FileStore/thinc"
import os; print("Exists:", os.path.exists(path))


In [0]:
%pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.7.1/en_core_web_lg-3.7.1-py3-none-any.whl